# K-means clustering

NP hard problem, to cluster items into groups with minimum distance to group centroids.
K-means is a greedy algorithm - finding local optimum.

```
function kmeans(k, points) is
    centroids ← list of k starting centroids, random or intelligent (k-means++)
    converged ← false

    while not converged false do
        // Create empty clusters
        clusters ← list of k empty lists

        // Assign each point to the nearest centroid
        foreach point in points
            point_centroids_distances = pointDistances(point, centroids)
            c = idxOfMinDistanceCentroid(point_centroids_distances)
            clusters[c].append(point)

        // Recalculate centroids as the mean of each cluster
        newCentroids = calculateCentroids(clusters)
        
        // Check for convergence
        if newCentroids == centroids THEN
            converged ← true
        else
            centroids ← newCentroids

    return clusters
```

## Imperative program

In [211]:
points = [ (0,0), (0,1), (1,0), (1,1), (10,10),(11,10),(10,11),(11,11) ]


In [226]:
from math import sqrt

def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return sqrt( (x2-x1)**2 + (y2-y1)**2 )

distance([0,0], [3,4])

5.0

In [230]:
def calc_centroid(cluster):
    return (
        sum(c[0] for c in cluster) / len(cluster),
        sum(c[1] for c in cluster) / len(cluster)
    )

calc_centroid([(0,0), (4,2)])

(2.0, 1.0)

In [231]:
def kmeans(k, points):
    converged = False
    centroids = points[:k]
    
    while converged == False:
        clusters = [[] for _ in range(k)]    

        for p in points:
            distances = [distance(p, c) for c in centroids]
            min_idx = distances.index(min(distances))
            clusters[min_idx].append(p)

        newCentroids = [calc_centroid(c) for c in clusters]

        if all([x == y for x,y in zip(centroids, newCentroids)]):
            converged = True
        else:
            centroids = newCentroids

    return centroids
    
kmeans(2, points)

[(0.5, 0.5), (10.5, 10.5)]

In [233]:
%timeit kmeans(2, points)

61 μs ± 7.09 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Torch

In [257]:
from torch import tensor, float

points_t = tensor([
    [0,0],
    [0,1],
    [1,0],
    [1,1],
    [10,10],
    [11,10],
    [10,11],
    [11,11],
    [30,30],
    [40,40]
], dtype=float).cuda()

input:
- Points are [N, D] , N points of 2 Dims
- Centroids are [K, D],  K centroids of 2 Dims

desired output:
- point-centroid distances, i.e. shape [K, N]

how?   
[K,D] ? [N,D] =?=> [K,N]
- expand on K,N: [K, 1, D] - [1, N, D] => [K, N, D] 
- reduce on D:   [K, N, D] => [K,N]

Using [None] to unsqueeze.
- Points [1, N, D] means we're treating Nx2D points as one batch, and we will broadcast (sort-of duplicate) this batch to match other matrix dimension
- Centroids [K, 1, D] means we're treating single 2D centroid as broadcasting (duplicating) target
Together, we mean take all points for each centroid, and broadcast each centroid to each point.

sum(2) is aggregate reduction on the last dimension

In [258]:
import torch

def distance_t(ps, cs):
    return ((ps.unsqueeze(0) - cs.unsqueeze(1)).pow(2)).sum(2).sqrt()
    #return ((cs.[:,None] - ps[None]).pow(2)).sum(2).sqrt()

p = tensor([
    [0,0],
    [1,1],
    [2,2]
]).cuda()
c = tensor([
    [1,0],
    [3,3]
]).cuda()

print(distance_t(p, c)), print(torch.cdist(c.float(), p.float(), 2))

tensor([[1.0000, 1.0000, 2.2361],
        [4.2426, 2.8284, 1.4142]], device='cuda:0')
tensor([[1.0000, 1.0000, 2.2361],
        [4.2426, 2.8284, 1.4142]], device='cuda:0')


(None, None)

Key insight is the `index_add_` method, capable of adding with a masked (with indexes) source

In [261]:
import torch

def newcentroids_t(points: tensor, c_idxs: tensor, k) -> tensor :
    counts, sums = torch.zeros(k, dtype=torch.float).cuda(), torch.zeros((k, 2), dtype=torch.float).cuda()
    counts.index_add_(0, c_idxs, torch.ones(points.shape[0]).cuda())
    sums.index_add_(0, c_idxs, points)
    return sums/counts.clamp_min(1).unsqueeze(1)    


newcentroids_t(points_t, tensor([1,1,1,1,1,1,1,1,0,0]).cuda(), 2)

tensor([[35.0000, 35.0000],
        [ 5.5000,  5.5000]], device='cuda:0')

In [4]:
def get_indexes(distances: tensor, k):
    idx = distances.argmin(0)
    u = idx.unique()
    if len(u) < k:
        for i in range(k):
            if not i in u:
                furthest = distances.min(0).values.argsort(descending=True)[0]       
                idx[furthest] = i

    return idx
    

In [262]:
def kmean_t(k, points: tensor, max_iterations=100):
    centroids = points[torch.randperm(len(points))[:k]]
    
    for _ in range(max_iterations):
        distances = distance_t(points, centroids)
        point_to_centroid_idx = get_indexes(distances, k)
        newCentroids = newcentroids_t(points, point_to_centroid_idx, k)

        if torch.all(centroids == newCentroids):
            return centroids
        else:
            centroids = newCentroids

    raise StopIteration()


kmean_t(4, points_t)

tensor([[ 0.0000,  0.5000],
        [10.5000, 10.5000],
        [35.0000, 35.0000],
        [ 1.0000,  0.5000]], device='cuda:0')

In [263]:
%timeit kmean_t(3, points_t)

6.46 ms ± 1.84 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
